# IBM Capstone Project 

In [10]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import json 
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup
from urllib.request import urlopen
import ssl
import csv

from geopy.geocoders import Nominatim 
# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Exploring City Data with the Four Square API
### Where is the best place to open a restuarant in London?


Where in London would it be best to open a new restuarant? 

This notebook leverages the four square API to explore neighborhood's in London. To explore this problem we need to segment London neigborhood data the data appropriatley. This entails segmenting by the average income of an area, what types of restuarants are popular (historically), and by how many restuarants there are in an area. We can then map the restaurants by location (latitude and longtitude) and then segment the restuarants by type. In this analysis the following questions will be answered; what restuarants recieve the best reviews? what restuarants are most popular, and what part of london are they in? Is there a type of restaurant that is consistently more popular than others? 

**Business Problem:**
This analysis will help anyone that wants to open up a restaurant in London and wants to know if it is a viable option where they intend on opening.


In [4]:
# Get longitude and latitude for London
address = 'London, London'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London is 51.5073219, -0.1276474.


In [17]:
# Set up Foursquare
CLIENT_ID = 'K2D5QVKQ0QPUPIKECQU2JIB3CHVFT4ZMTJQC5WVTRNYTDD5Z'
CLIENT_SECRET = '5ZVFLDC04W0W5SHCBZ11MDRG0MELPIER3QIFUOAWFAIZKJNK'
VERSION = '20200808' # Foursquare API version
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K2D5QVKQ0QPUPIKECQU2JIB3CHVFT4ZMTJQC5WVTRNYTDD5Z
CLIENT_SECRET:5ZVFLDC04W0W5SHCBZ11MDRG0MELPIER3QIFUOAWFAIZKJNK


In [18]:
search_query = 'Restaurant'
radius = 50000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [19]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=K2D5QVKQ0QPUPIKECQU2JIB3CHVFT4ZMTJQC5WVTRNYTDD5Z&client_secret=5ZVFLDC04W0W5SHCBZ11MDRG0MELPIER3QIFUOAWFAIZKJNK&ll=51.5073219,-0.1276474&v=20200808&query=Restaurant&radius=50000&limit=500'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed78a3ba2e538001b8e3b95'},
 'response': {'venues': [{'id': '4af41e26f964a52024f021e3',
    'name': 'Portrait Restaurant',
    'location': {'address': "St. Martin's Pl",
     'crossStreet': 'Trafalgar Sq',
     'lat': 51.509259272728734,
     'lng': -0.12820160310541204,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.509259272728734,
       'lng': -0.12820160310541204}],
     'distance': 219,
     'postalCode': 'WC2N 4JJ',
     'cc': 'GB',
     'city': 'London',
     'state': 'Greater London',
     'country': 'United Kingdom',
     'formattedAddress': ["St. Martin's Pl (Trafalgar Sq)",
      'London',
      'Greater London',
      'WC2N 4JJ',
      'United Kingdom']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'venue

In [21]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/Users/joseph_arber/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4af41e26f964a52024f021e3,Portrait Restaurant,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1591184204,False,St. Martin's Pl,Trafalgar Sq,51.509259,-0.128202,"[{'label': 'display', 'lat': 51.50925927272873...",219,WC2N 4JJ,GB,London,Greater London,United Kingdom,"[St. Martin's Pl (Trafalgar Sq), London, Great...",130427500,NaN
1,4d440bf1c3e5f04d0a219620,Bill's Restaurant,"[{'id': '52e81612bcbc57f1066b7a05', 'name': 'E...",v-1591184204,False,28 St Martin's Courtyard,Upper St Martin's Ln,51.512555,-0.126600,"[{'label': 'display', 'lat': 51.51255465283461...",587,WC2E 9AB,GB,Covent Garden,Greater London,United Kingdom,[28 St Martin's Courtyard (Upper St Martin's L...,NaN,NaN
2,4cf8f4e0d76da35d62e8c92c,River Restaurant,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",v-1591184204,False,The Savoy,NaN,51.510180,-0.120549,"[{'label': 'display', 'lat': 51.51017988179497...",585,WC2R 0EU,GB,London,Greater London,United Kingdom,"[The Savoy, London, Greater London, WC2R 0EU, ...",NaN,NaN
3,4ed23e3b2c5bc35059276f87,London Chinatown Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1591184204,False,27 Gerrard St,NaN,51.511507,-0.131810,"[{'label': 'display', 'lat': 51.51150741274579...",547,W1D 6JN,GB,London,Greater London,United Kingdom,"[27 Gerrard St, London, Greater London, W1D 6J...",NaN,NaN
4,4c39bcafedba9521cf6cd625,New Loon Fung Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1591184204,False,42-44 Gerrard St,NaN,51.512039,-0.130371,"[{'label': 'display', 'lat': 51.51203891754673...",557,W1D 5QG,GB,London,Greater London,United Kingdom,"[42-44 Gerrard St, London, Greater London, W1D...",NaN,NaN


In [34]:
Neighbourhoods = list(dataframe.columns.values)
Neighbourhoods = Neighbourhoods[5:]
print(Neighbourhoods)

['location.address', 'location.crossStreet', 'location.lat', 'location.lng', 'location.labeledLatLngs', 'location.distance', 'location.postalCode', 'location.cc', 'location.city', 'location.state', 'location.country', 'location.formattedAddress', 'venuePage.id', 'location.neighborhood']


In [22]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Portrait Restaurant,Café,St. Martin's Pl,Trafalgar Sq,51.509259,-0.128202,"[{'label': 'display', 'lat': 51.50925927272873...",219,WC2N 4JJ,GB,London,Greater London,United Kingdom,"[St. Martin's Pl (Trafalgar Sq), London, Great...",NaN,4af41e26f964a52024f021e3
1,Bill's Restaurant,English Restaurant,28 St Martin's Courtyard,Upper St Martin's Ln,51.512555,-0.126600,"[{'label': 'display', 'lat': 51.51255465283461...",587,WC2E 9AB,GB,Covent Garden,Greater London,United Kingdom,[28 St Martin's Courtyard (Upper St Martin's L...,NaN,4d440bf1c3e5f04d0a219620
2,River Restaurant,Breakfast Spot,The Savoy,NaN,51.510180,-0.120549,"[{'label': 'display', 'lat': 51.51017988179497...",585,WC2R 0EU,GB,London,Greater London,United Kingdom,"[The Savoy, London, Greater London, WC2R 0EU, ...",NaN,4cf8f4e0d76da35d62e8c92c
3,London Chinatown Restaurant,Chinese Restaurant,27 Gerrard St,NaN,51.511507,-0.131810,"[{'label': 'display', 'lat': 51.51150741274579...",547,W1D 6JN,GB,London,Greater London,United Kingdom,"[27 Gerrard St, London, Greater London, W1D 6J...",NaN,4ed23e3b2c5bc35059276f87
4,New Loon Fung Chinese Restaurant,Chinese Restaurant,42-44 Gerrard St,NaN,51.512039,-0.130371,"[{'label': 'display', 'lat': 51.51203891754673...",557,W1D 5QG,GB,London,Greater London,United Kingdom,"[42-44 Gerrard St, London, Greater London, W1D...",NaN,4c39bcafedba9521cf6cd625


In [33]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=15,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map